In [ ]:
%load_ext autoreload
%autoreload 2

### IMPORTING FILES FROM THE PROJECT ROOT.

In [ ]:
import numpy as np

#### --- MAKING ROOT PROJECT FOLDER VISIBLE AT SCRIPT LEVEL ---

##
## --- Finding root project folder ---
##     (needed to import settings)
##
import os, sys
def find_project_root(beacon="ROOT_BEACON"):
    current_path = os.getcwd()
    
    while True:
        if os.path.isfile(os.path.join(current_path, beacon)):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        if parent_path == current_path:
            raise RuntimeError(f"Unable to find project root.\nNo '{beacon}' file found in any parent directory.")
        
        current_path = parent_path

# --- Find and set project root ---
project_root = find_project_root()

os.chdir(project_root)
####                    --- end ---


#### IMPORTING FROM ROOT PROJECT FOLDER 
from Utils import ArrayStorageCompressor, ArrayDataManager, ArraySettings
from Utils import ArrayGenerator

### CALCULATE A, B COEFFICIENTS FOR THE FORMULA   $x_i = a * b^i$

In [ ]:
a = ArraySettings.ARRAY_START_KEY
b = np.exp((np.log(ArraySettings.ARRAY_END_KEY)-np.log(ArraySettings.ARRAY_START_KEY))/(ArraySettings.NUMBER_OF_SAMPLES-1))

print(a, b)

### CALCULATE KEYS BASED THE GEOMETRIC SERIES

In [ ]:
x_i = np.array([int(np.floor(a*np.pow(b, i))) for i in range(ArraySettings.NUMBER_OF_SAMPLES)])
print(x_i)

### GENERATE AND UPLOAD DATA ON ARRAY MANAGER

In [ ]:
%%time

arrayManager = ArrayDataManager.ArraySampleContainer()
parameters_create_function = ArrayGenerator.initialize_creation_parameters_environment()

for i in range(ArraySettings.NUMBER_OF_SAMPLES):
    n, m, rep, dtype, seeds = parameters_create_function(i, x_i[i])
    creation_arguments = {
        'n': n,
        'm': m,
        'rep': rep,
        'dtype': dtype,
        'seeds': seeds
    }
    
    arrayManager.update({
        int(x_i[i]): 
            ArrayDataManager.ArraySample(
                sample= ArrayGenerator.sample(
                            **creation_arguments
                        ),
                creation_arguments= ArrayDataManager.ArraySampleCreationArguments(
                        **creation_arguments
                    ),
                variability = int(x_i[i])
            )
    })

print(f"[Status] Generated {len(arrayManager.keys())} samples over {ArraySettings.NUMBER_OF_SAMPLES} keys.")
print(f"[Memory] Total array data size: {arrayManager.estimate_data_size_MB():.3f} MB")

### SAVING ARRAYS ON LOCAL FILE

In [ ]:
storage_folder_path = ArraySettings.CREATE_ARRAY_STORAGE_FOLDER()
ArrayStorageCompressor.writeOnFile(
            data= arrayManager,
            path= os.path.join(storage_folder_path, ArraySettings.GET_COMPRESSED_ARRAY_FILE_NAME)
    )
    
ArrayStorageCompressor.writeOnFile(arrayManager.get_creation_arguments(), os.path.join(storage_folder_path, ArrayGenerator.ARRAY_GENERATION_FILE), as_json= True)
print(f"Array generation file location: { os.path.join(storage_folder_path, ArrayGenerator.ARRAY_GENERATION_FILE)}")